<a href="https://colab.research.google.com/github/riya25bet10051-sketch/Alzymer-detector-model/blob/main/Copy_of_Alzymer_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from  google.colab import files
uploded = files.upload()

In [ ]:
! unzip "archive (1).zip"

In [ ]:
!pip install datasets pandas pillow tensorflow

In [ ]:
import os
from datasets import load_dataset
import tensorflow as tf
import numpy as np
from PIL import Image
import io

# 1. Define the paths to your unzipped files
# Based on your screenshot, the folder name is "Alzheimer MRI Disease Classification Dataset"
base_path = "Alzheimer MRI Disease Classification Dataset/Data"
train_file = os.path.join(base_path, "train-00000-of-00001*") # * captures the full hash
test_file = os.path.join(base_path, "test-00000-of-00001*")

print("Loading dataset from files...")

# 2. Load using the Hugging Face datasets library
# We use shell globbing (the *) to match the long file names automatically
dataset = load_dataset("parquet", data_files={"train": train_file, "test": test_file})

print("Dataset loaded successfully!")
print(dataset)

In [ ]:
IMG_SIZE = 128
BATCH_SIZE = 32

def preprocess_data(examples):
    # This function processes a batch of raw data
    images = []
    for img_data in examples['image']:
        # Check if image is already a dict (common in some parquet exports) or raw bytes
        if isinstance(img_data, dict) and 'bytes' in img_data:
            img_bytes = img_data['bytes']
        else:
            img_bytes = img_data # Assume it's direct bytes or PIL object

        # Convert bytes to PIL Image
        if not isinstance(img_bytes, Image.Image):
            image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        else:
            image = img_bytes.convert("RGB") # It's already a PIL image

        # Resize and Normalize
        image = image.resize((IMG_SIZE, IMG_SIZE))
        image = np.array(image) / 255.0 # Normalize to 0-1
        images.append(image)

    return {"pixel_values": images, "label": examples['label']}

# Apply the preprocessing to the dataset
print("Processing images... this might take a minute.")
processed_dataset = dataset.map(preprocess_data, batched=True, batch_size=32, remove_columns=["image"])

# Convert to TensorFlow compatible format
train_ds = processed_dataset["train"].to_tf_dataset(
    columns="pixel_values", label_cols="label", batch_size=BATCH_SIZE, shuffle=True
)
test_ds = processed_dataset["test"].to_tf_dataset(
    columns="pixel_values", label_cols="label", batch_size=BATCH_SIZE, shuffle=False
)

print("Data is ready for training.")

In [ ]:
from tensorflow.keras import layers, models

def create_model():
    model = models.Sequential([
        # Input Layer
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),

        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5), # Helps prevent overfitting

        # Output Layer (4 classes: Mild, Moderate, Non, Very Mild)
        layers.Dense(4, activation='softmax')
    ])

    return model

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train for 10 epochs
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

# Plot accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize lists to store results
y_true = []
y_pred = []

print("Generating predictions... this may take a moment.")

# Iterate through the test dataset batch by batch
for images, labels in test_ds:
    # Make predictions on the current batch
    preds = model.predict(images, verbose=0)

    # Convert probabilities to class index (0, 1, 2, or 3)
    predicted_classes = np.argmax(preds, axis=1)

    # Store the results
    y_pred.extend(predicted_classes)
    y_true.extend(labels.numpy())

print("Predictions complete.")

In [ ]:

class_names = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)

plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Get a batch of images
for images, labels in test_ds.take(1):
    predictions = model.predict(images)
    predicted_class = np.argmax(predictions[0])
    true_class = labels[0].numpy()

    classes = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']

    print(f"True Class: {classes[true_class]}")
    print(f"Predicted Class: {classes[predicted_class]}")

    # Show the image
    plt.imshow(images[0])
    plt.axis('off')
    plt.show()